# Gradient Descent and Linear Regression with Pytorch

## Introduction to Linear Regression

Suppose the temperature, rainfall and humidity was recorded along with the yield of apples and oranges in five regions as shown below

<br />

| Region | Temp. (F) | Rainfall (mm) | Humidity (%) | Apples (ton) | Oranges (ton) |
|:------:|:---------:|:-------------:|:------------:|:------------:|:-------------:|
| Kanto | 73 | 67 | 43 | 56 | 70 |
| Johto | 91 | 88 | 64 | 81 | 101 |
| Hoenn | 87 | 134 | 58 | 119 | 133 |
| Sinnoh | 102 | 43 | 37 | 22 | 37 |
| Unova | 69 | 96 | 70 | 103 | 109 |

<br />

Using the average temperature, rainfall and humidity, we'll create a linear regression model which predicts the crop yields for apples and oranges

The yield of apple and oranges can be estimated as a weighted sum of the input variables   

> apple_yield = $w_11 * temp + w_12 * rainfall + w_13 * humidity + b1$  
> orange_yield = $w_21 * temp + w_22 * rainfall + w_23 * humidity + b2$

In [2]:
import numpy as np
import torch

## Training Data

We can represent the inputs as a 2d-array with each column representing the features

In [79]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [80]:
# Targets (apple, orange)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Convert numpy array to tensors - since we will be training our model using PyTorch

In [81]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [121]:
# Initialize weight and bias with random numbers
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[ 0.0985, -1.2851,  1.4055],
        [ 0.4298,  0.8026,  0.2298]], requires_grad=True)
tensor([-0.1701, -1.0593], requires_grad=True)


In [122]:
# create a simple linear regression model
def model(x):
    return inputs @ w.t() + b

where `@` represnt matrix multiplication in PyTorch and `.t` return the transpose of a tensor

In [123]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-18.6428,  93.9705],
        [-14.3408, 123.3872],
        [-82.2823, 157.2096],
        [  6.6233,  85.7918],
        [-18.3558, 121.7323]], grad_fn=<AddBackward0>)


## Loss Function

We evaluate our model performance by computing the mean squared error 

In [124]:
(preds - targets)

tensor([[ -74.6428,   23.9705],
        [ -95.3408,   22.3872],
        [-201.2823,   24.2096],
        [ -15.3767,   48.7918],
        [-121.3558,    2.7323]], grad_fn=<SubBackward0>)

In [125]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return (torch.sum(diff ** 2)) / diff.numel()

In [126]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(7418.9624, grad_fn=<DivBackward0>)


This means that on average each element in our predictions differs from the actual target by the square root of the loss

In [127]:
# Compute gradients
loss.backward()

Each element in the `w.grad` represent the loss with respect to the corresponding weight in `w`

Since the derivate of the loss w.r.t. a particular weight indicate the rate of change of the loss as the weight is slightly changed, we can use this concept to reduce our loss. 

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly. If the gradient is positive, then the weight decreases. However, if the gradient is negative, then the weight decreases.

In [128]:
print(w)
print(w.grad)

tensor([[ 0.0985, -1.2851,  1.4055],
        [ 0.4298,  0.8026,  0.2298]], requires_grad=True)
tensor([[ -8315.6934, -10534.8477,  -6009.9331],
        [  2211.7212,   1836.1052,   1172.8446]])


In [129]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we do not modify the weights by a very large amount. 

Computing derivatives is an expensive operation. We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases. 

In [130]:
# Let's verify that the loss is actually lower
print("Initial loss:", loss)

pred = model(inputs)

loss = mse(pred, targets)
print("Updated loss:", loss)

Initial loss: tensor(7418.9624, grad_fn=<DivBackward0>)
Updated loss: tensor(5361.0703, grad_fn=<DivBackward0>)


We reset the gradients to zero by calling the .zero_() method. This is because PyTorch accumulates gradients and we need to avoid the existing gradients being added to the new gradients.  

In [136]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradients descent optimization algorithm. Thus, we can train the model using the following steps:

1. Generate Predictions
2. Calculate the loss
3. Compute gradients with respect to the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [137]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -2.9287,  90.6212],
        [  6.3445, 119.0079],
        [-57.4442, 152.1445],
        [ 21.8600,  82.3122],
        [  1.7035, 117.6223]], grad_fn=<AddBackward0>)


In [142]:
# Calculate the loss 
loss = mse(preds, targets)
print(loss)

tensor(5361.0703, grad_fn=<DivBackward0>)


In [143]:
# Compute the gradients 
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-6689.9653, -8778.3828, -4928.3169],
        [ 1867.2826,  1469.5680,   945.9412]])
tensor([-82.2930,  20.3416])


In [144]:
# Adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [145]:
# New weights and biases
print(w)
print(b)

tensor([[ 0.2486, -1.0920,  1.5149],
        [ 0.3890,  0.7696,  0.2086]], requires_grad=True)
tensor([-0.1683, -1.0598], requires_grad=True)


In [146]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(3970.1011, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights an biases using the gradient multiple times. Each iteration is an called epoch

In [152]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [153]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(374.7664, grad_fn=<DivBackward0>)


In [154]:
# Prediction
preds

tensor([[ 63.2298,  73.1528],
        [ 95.7874,  97.7997],
        [ 78.0563, 134.9118],
        [ 57.4500,  54.0936],
        [104.2895, 103.9684]], grad_fn=<AddBackward0>)

In [155]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [158]:
# Train for an extra 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [159]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(154.7007, grad_fn=<DivBackward0>)


## Linear Regression using Pytorch Built-ins

In [160]:
import torch.nn as nn

In [161]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43],
                   [91, 87, 65], 
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                   dtype='float32')
                   
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119], 
                    [57, 69], 
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [162]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

We'll create a TensorDataset, which allows to access rows from `inputs` and `targets` as tuples, and provides a standard API for working with many different types of datasets in PyTorch

In [163]:
from torch.utils.data import TensorDataset

In [198]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [199]:
from torch.utils.data import DataLoader

In [200]:
# Define the data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [201]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 88., 134.,  59.],
        [ 87., 135.,  57.],
        [ 73.,  66.,  44.],
        [ 68.,  97.,  70.],
        [103.,  43.,  36.]])
tensor([[118., 132.],
        [118., 134.],
        [ 57.,  69.],
        [102., 120.],
        [ 20.,  38.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.linear class from PyTorch, which does it automatically.

In [206]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0212, -0.5354,  0.2916],
        [-0.4104, -0.3464, -0.5111]], requires_grad=True)
Parameter containing:
tensor([ 0.1150, -0.0735], requires_grad=True)


In [208]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0212, -0.5354,  0.2916],
         [-0.4104, -0.3464, -0.5111]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1150, -0.0735], requires_grad=True)]

In [210]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -21.6755,  -75.2224],
        [ -26.4155, -100.6177],
        [ -52.8789, -111.8453],
        [  -9.9608,  -75.7447],
        [ -29.4154,  -97.4256],
        [ -21.1189,  -75.2864],
        [ -25.5886, -100.7823],
        [ -52.5662, -112.7668],
        [ -10.5173,  -75.6807],
        [ -29.1450,  -97.5262],
        [ -20.8486,  -75.3870],
        [ -25.8589, -100.6817],
        [ -53.7059, -111.6807],
        [ -10.2311,  -75.6441],
        [ -29.9720,  -97.3616]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use built-in loss function mse_loss

In [213]:
# Import nn.functional
import torch.nn.functional as F

In [214]:
# Define loss function
loss_fn = F.mse_loss

In [215]:
# Compute the loss for the current predictions
loss_fn(preds, targets)

tensor(24678.2520, grad_fn=<MseLossBackward0>)

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`, where SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group

In [216]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [223]:
for xb, yb in train_dl:
    print(xb, yb)

tensor([[ 91.,  87.,  65.],
        [ 69.,  96.,  70.],
        [ 88., 134.,  59.],
        [ 92.,  87.,  64.],
        [ 73.,  66.,  44.]]) tensor([[ 80., 102.],
        [103., 119.],
        [118., 132.],
        [ 82., 100.],
        [ 57.,  69.]])
tensor([[ 74.,  66.,  43.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.],
        [ 87., 135.,  57.],
        [102.,  43.,  37.]]) tensor([[ 57.,  69.],
        [104., 118.],
        [ 21.,  38.],
        [118., 134.],
        [ 22.,  37.]])
tensor([[103.,  43.,  36.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 68.,  97.,  70.],
        [ 87., 134.,  58.]]) tensor([[ 20.,  38.],
        [ 81., 101.],
        [ 56.,  70.],
        [102., 120.],
        [119., 133.]])


In [231]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeact for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred =  model(xb)
            
            # 2. Compute the loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress at the end of every 10th epoch 
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [232]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 30.9586
Epoch [20/100], Loss: 19.2758
Epoch [30/100], Loss: 19.4214
Epoch [40/100], Loss: 11.1591
Epoch [50/100], Loss: 20.2170
Epoch [60/100], Loss: 17.7810
Epoch [70/100], Loss: 16.1689
Epoch [80/100], Loss: 16.4028
Epoch [90/100], Loss: 14.0094
Epoch [100/100], Loss: 9.7674


In [234]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1723,  70.9516],
        [ 82.3367,  97.6165],
        [117.2547, 139.1190],
        [ 21.9237,  40.2652],
        [101.7591, 111.8979],
        [ 55.9636,  69.8275],
        [ 82.2329,  97.1398],
        [117.5789, 139.4152],
        [ 23.1324,  41.3893],
        [102.8641, 112.5452],
        [ 57.0686,  70.4749],
        [ 81.1280,  96.4924],
        [117.3585, 139.5957],
        [ 20.8188,  39.6179],
        [102.9679, 113.0220]], grad_fn=<AddmmBackward0>)

In [235]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])